# Intermediate Machine Learning: Assignment 5

**Deadline**

Assignment 5 is due Wednesday, December 6 by 11:59pm. Late work will not be accepted as per the course policies (see the Syllabus and Course policies on Canvas).

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged.

You should start early so that you have time to get help if you're stuck. The drop-in office hours schedule can be found on Canvas. You can also post questions or start discussions on Ed Discussion. The assignment may look long at first glance, but the problems are broken up into steps that should help you to make steady progress.

**Submission**

Submit your assignment as a pdf file on Gradescope, and as a notebook (.ipynb) on Canvas. You can access Gradescope through Canvas on the left-side of the class home page. The problems in each homework assignment are numbered. Note: When submitting on Gradescope, please select the correct pages of your pdf that correspond to each problem. This will allow graders to more easily find your complete solution to each problem.

To produce the .pdf, please do the following in order to preserve the cell structure of the notebook:

Go to "File" at the top-left of your Jupyter Notebook
Under "Download as", select "HTML (.html)"
After the .html has downloaded, open it and then select "File" and "Print" (note you will not actually be printing)
From the print window, select the option to save as a .pdf

**Topics**

 * Policy iteration
 * RNNs and GRUs

This assignment will also help to solidify your Python skills.

## Problem 1: Reinforcement Learning: Policy Iteration (10 Points)

During class, we talked about the policy iteration algorithm, which is a tabular method that estimates the optimal policy using iterative policy evaluation. 
More specifically, at each policy evaluation step, we follow a policy $\pi$ and update the value function $V(s)$ for each state $s$ as 

$$
V(s) \gets \sum_{s', r} p(s', r|s, \pi(s))[r + \gamma V(s')];
$$

at each policy improvement step, we improve the current policy $\pi(s)$ following the updated value function: 

$$
\pi(s) \gets \text{argmax}_{a}\sum_{s',r} p(s', r|s,a)[r+\gamma V(s')].
$$

Following this iteration of policy evaluation and policy improvement, we can obtain a sequence of monotonically improving policies and value functions. In other words, each policy is guaranteed to be a strict improvement over the previous one unless it is an optimal policy, i.e., $\pi = \pi^*$.

Since a finite MDP (Markov Decision Process) has only a finite number of policies, this process eventually converges to an optimal policy $\pi^*$ and the corresponding optimal value function in a finite number of iterations.

However, it is not entirely clear *a priori* why we are able to constantly improve our policy $\pi$ following this alternating process. In this question, we walk you through a proof that the policy improves in each iteration of the algorithm. 

1. Suppose we have a policy $\pi$ and a monotonically improved policy $\pi'$, such that for all states $s \in S$, we have 

$$
Q_{\pi}(s, \pi'(s)) \ge V_{\pi}(s).
$$

Show that the value function $V_{\pi'}$ dominates $V_{\pi}$, i.e., for all states $s \in S$

$$
V_{\pi'}(s) \ge V_{\pi}(s).
$$

*hint: Consider expanding the value functions iteratively.*

Define the following terms:
* $f(s) = V_{\pi'}(s) - V_{\pi}(s)$
* $g(s) = V_{\pi'}(s) - Q_{\pi}(s,\pi'(s))$
The condition given by the problem is equivalent to 
$$
f(s) \ge g(s),  \forall s
$$
And my goal is to prove:
$$
f(s) \ge 0 
$$

Now, let's begin the proof:

Recall that:
$$
V_{\pi'}(s) 
= r(s,\pi'(s)) + \gamma \mathbb E_{s_1\sim P (\cdot|s,\pi'(s))} V_{\pi'} (s_1)  \\
= r(s,\pi'(s)) + \gamma \int V_{\pi'} (s_1) p(s_1|s,\pi'(s)) ds_1
\\
Q_{\pi}(s,\pi'(s)) 
= r(s,\pi'(s)) + \gamma \mathbb E_{s_1\sim P (\cdot|s,\pi'(s))} V_{\pi} (s_1) \\
= r(s,\pi'(s)) + \gamma \int V_{\pi} (s_1) p(s_1|s,\pi'(s)) ds_1
$$

Now we can have:

$$
f(s) = V_{\pi'}(s) - V_\pi(s) \\
\ge g(s) = V_{\pi'} (s) - Q_\pi(s,\pi'(s)) \\
= r(s,\pi'(s)) +  \gamma\int V_{\pi'} (s_1) p(s_1|s,\pi'(s)) ds_1\\ - r(s,\pi'(s)) +\gamma \int V_{\pi} (s_1) p(s_1|s,\pi'(s)) ds_1 \\
= \gamma\int [V_{\pi'} (s_1) - V_{\pi} (s_1)] p(s_1|s,\pi'(s)) ds_1 \\
= \gamma\int f(s_1) p(s_1|s,\pi'(s)) ds_1 \\
= \gamma\mathbb E_{s_1\sim P_1(\cdot|s,\pi')} f(s_1)
$$
where $P_t(\cdot|s,\pi')$ is the probablity distribution of the state $s_t$ of starting at $s$ and follow the policy $\pi'$ for $t$ step. 
Essentially, the previous mathematical computation tells us that 
$$
f(s) \ge \gamma \mathbb E_{s_1\sim P_1(\cdot|s,\pi')} f(s_1)
$$


By mathematical induction,
$$
f(s) \ge \gamma^t \mathbb E_{s_t\sim P_t(\cdot|s,\pi')} f(s_t),\quad \forall t\in\mathbb N
$$

Now, let's examine $s^* = \text{argmin}_{s} f(s)$, we have that $\forall t\in\mathbb N$
$$
f(s^*) \ge \gamma^t \mathbb E_{s_t\sim P_t(\cdot|s^*,\pi')} f(s_t) \ge \gamma^t \mathbb E_{s_t\sim P_t(\cdot|s^*,\pi')} f(s^*) = \gamma^t f(s^*)
$$

Letting $t\to\infty$, the RHS of the above equation goes to $0$. Therefore
$$
f(s^*) \ge 0
$$
And we know for a fact that $f(s)\ge f(s^*)\ge0$. The proof is finished. 




2. Apply the conclusion of part 1 to the policy improvement step to show that it leads to a sequence of monotonically improving policies. In other words, show that if $\pi \neq \pi^*$, the next round policy $\pi'$ under the policy iteration algorithm satisfies $V_{\pi'}(s) > V_{\pi}(s)$ for some state $s\in S$.

Let's use proof by contradiction: prove that given $V_{\pi'}(s) = V_\pi(s)$, then $\forall s$, $\pi'=\pi$. 

Proof. 
given that $V_{\pi'}(s) = V_\pi(s)$ $\forall s$, then 

$$
V_{\pi}(s) = V_{\pi'}(s) = \sum_{s', r} p(s', r|s, \pi'(s))[r + \gamma V^{\pi'}(s')] = \sum_{s', r} p(s', r|s, \pi'(s))[r + \gamma V^{\pi}(s')]
$$


Since the update rule dictates that
$$\pi'(s) = \text{argmax}_a \sum_{s',r} p(s', r|s,a)[r+\gamma V^{\pi}(s')]$$
we know that 
$$
\sum_{s', r} p(s', r|s, \pi'(s))[r + \gamma V^{\pi}(s')] = \max_a \sum_{s', r} p(s', r|s, a)[r + \gamma V^{\pi}(s')]
$$

In a summary, we now have
$$
V_{\pi}(s) 
= V_{\pi'}(s) \\
= \sum_{s', r} p(s', r|s, \pi'(s))[r + \gamma V^{\pi'}(s')] \\
= \sum_{s', r} p(s', r|s, \pi'(s))[r + \gamma V^{\pi}(s')]  \\
= \max_a \sum_{s', r} p(s', r|s, a)[r + \gamma V^{\pi}(s')] \\
$$
This is the Bellman equation that gives us the optimality of $\pi$. 
And the proof by contradition is finished. 

## Problem 2: Elephants Can Remember (20 points)

Text generation is a common task in Natural Language Processing (NLP), where, given an initial text as a prompt, the model will produce human-like text that continues the prompt. Over the past years, transformer-based models (like GPT-3) have taken over the domain of text generation. In this problem, let's take a step back and focus on the earlier sequence models for text generation: Vanilla Recurrent Neural Networks (RNNs) and Recurrent Neural Networks with Gated Recurrent Units (GRUs). The models in this part of the assignment will be character-based models, trained on an extract of the book "Elephants Can Remember" by Agatha Christie. To reduce the size of our vocabulary, the text is pre-processed by converting the letters to lower case and removing numbers. The code below shows some information about our training and test set. All the necessary files for this problem are available through Canvas, under the file name "q2_data", a compressed folder.

In [2]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import random
  
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation
from keras.layers import GRU, SimpleRNN

In [3]:
with open('q2_data/Agatha_Christie_train.txt', 'r') as file:
    train_text = file.read()
    
with open('q2_data/Agatha_Christie_test.txt', 'r') as file:
    test_text = file.read()

vocabulary = sorted(list(set(train_text + test_text)))
vocab_size = len(vocabulary)

# Dictionaries to go from a character to index and vice versa
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

In [5]:
# The first 500 characters of our training set
train_text[0:500]

'mrs. oliver looked at herself in the glass. she gave a brief, sideways look towards the clock on the mantelpiece, which she had some idea was twenty minutes slow. then she resumed her study of her coiffure. the trouble with mrs. oliver was--and she admitted it freely--that her styles of hairdressing were always being changed. she had tried almost everything in turn. a severe pompadour at one time, then a wind-swept style where you brushed back your locks to display an intellectual brow, at least'

In [6]:
print("The vocabulary contains", vocab_size, "characters")
print("The training set contains", len(train_text) ,"characters")
print("The test set contains", len(test_text) ,"characters")

The vocabulary contains 44 characters
The training set contains 262174 characters
The test set contains 7209 characters


### Problem 2.1: The Diversity of Language Models

Before jumping into coding, let's start with comparing the language models we will be using in this assigment.

1. Describe the differences between a Vanilla RNN and a GRU network. In your explanation, make sure you mention the issues with vanilla RNNs and how GRUs try to solve them.

GRU has an additional gates. The first gate is the reset gates, which transforms the hidden state to be used to generate context. the 2nd gate is the memory gate, which choose from the context and the hidden state to generate the new hidden state. 

2. Describe at least two advantages of a character based language model over a word based language model.

* character based model can have a simple one-hot embedding into vector space. 
* there is no computational bottleneck at the output of character based model

### Problem 2.2: Generating Text with the Vanilla RNN

The code below loads in a pretrained vanilla RNN model with two layers. The model is set up exactly like in the lecture slides (with tanh activation layers in the recurrent layers) with the addition of biases (intercepts) in every layer (i.e. the recurrent layer and the dense layer). The training process consisted of 30 epochs.

In [7]:
# load json and create model
json_file = open('q2_data/RNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

RNN_model = model_from_json(loaded_model_json)
RNN_model.load_weights("q2_data/RNN_model.h5")

In [8]:
# load in the weights and show summary
weights_RNN = RNN_model.get_weights()
RNN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Vanilla_RNN_1 (SimpleRNN)   (None, 100, 128)          22144     
                                                                 
 Vanilla_RNN_2 (SimpleRNN)   (None, 64)                12352     
                                                                 
 Dense_layer (Dense)         (None, 44)                2860      
                                                                 
 Softmax_layer (Activation)  (None, 44)                0         
                                                                 
Total params: 37,356
Trainable params: 37,356
Non-trainable params: 0
_________________________________________________________________


Finish the following function that uses a vanilla RNN architecture to generate text, given the weights of the RNN model, a text prompt, and the number of characters to return. The function should be completed by **only using numpy functions**. Use your knowledge of how every weight plays its role in the RNN architecture. Do not worry about the weight extraction part, this is already provided for you. The weight matrix $W_{xh1}$, for example, denotes the weight matrix to go from the input x to the first hidden state layer h1. The hidden states $h_1$ and $h_2$ are initialized to a vector of zeros. 

The embedding of each character has to be done by a one-hot encoding, where you will need the dictionaries defined in the introduction to go from a character to an index position.

In [11]:
weights_RNN[0].shape, weights_RNN[]

(44, 128)

In [19]:
def sample_text_RNN(weights, prompt, N):
    '''
    Uses a pretrained RNN to generate text, starting from a prompt, 
    only using the weights and numpy commands
            Parameters:
                    weights (list): Weights of the pretrained RNN model
                    prompt (string): Start of generated sentence
                    N (int): Length of output sentence (including prompt)
            Returns:
                    output_sentence (string): Text generated by RNN
    '''
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides

    # First Recurrent Layer 
    W_xh1 = weights[0].T 
    W_h1h1 = weights[1].T 
    b_h1 = np.expand_dims(weights[2], axis=1)

    # Second Recurrent Layer
    W_h1h2 = weights[3].T
    W_h2h2 = weights[4].T
    b_h2 = np.expand_dims(weights[5], axis=1)

    # Linear (dense) layer
    W_h2y = weights[6].T
    b_y = np.expand_dims(weights[7], axis=1)
    
    # Initiate the hidden states
    h1 = np.zeros((W_h1h1.shape[0], 1))
    h2 = np.zeros((W_h2h2.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    
    output_sentence = ''
    for i in range(N):
        
        if i < len(prompt):
            output_sentence += prompt[i]
        else:
            output_sentence += y
        
        # onehot embedding
        x = output_sentence[i]
        x = char_to_indices[x]
        x_one_hot = np.zeros((vocab_size, 1))
        x_one_hot[x] = 1
        x = x_one_hot
        
        # apply RNN
        h1 = np.tanh(np.dot(W_h1h1,h1) + np.dot(W_xh1,x) + b_h1)
        h2 = np.tanh(np.dot(W_h2h2,h2) + np.dot(W_h1h2,h1) + b_h2)
        y = np.dot(W_h2y,h2) + b_y
        y = np.exp(y) / np.sum(np.exp(y))
        
        # sample from the distribution p
        y = np.random.choice(range(vocab_size), p=y.ravel())
        
        # transform to character
        y = indices_to_char[y]
        
    return output_sentence

Test out your function by running the following code cell. Use it as a sanity check that your code is working. The generated text should not be perfect English, but at least you should be able to recognize some words.

In [20]:
print(sample_text_RNN(weights_RNN, 
                      'mrs. oliver looked at herself in the glass. she gave a brief, sideways look', 
                      1000))

mrs. oliver looked at herself in the glass. she gave a brief, sideways looked are somerough or your soppenilate, but they heare apting celing on he tagh there was at as elephants lyodd the, you wanted aok there it for apseancile because call coustil two menty. not," said mrs. oliver. "i'm ongen all thas moseird things he have rement?" "no. are mracwabach, and so have yes. corden you beart, argut abdear from once things that sent abluthord, are live something people? you lought whow you doel no that there. i'm not no," said daypressing to to cally, rading commint ot havenscrofn. then they'd. i ence faccuthere," said mrvatyer. you knem shad," said mrs. oliver. anywas, they was her. i meane?" "and it's it was afted to the wistsuse. had beel lings. "i have bely wigh rell really that have been ttrentle infieved i've been ceable things and crust of happen, helpersully wey-," said mrs. oliver, it's someable shos, i say, it bearny suanes forgetly you lust came well you werentiestly ele," said 

### Problem 2.3: Generating Text with the GRU

The code below loads in a pretrained GRU model. The model is set up exactly like in the lecture slides (with sigmoid activation layers for the gates and tanh activation layers in the recurrent layer). The model is trained for only 10 epochs.

In [21]:
# load json and create model
json_file = open('q2_data/GRU_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

GRU_model = model_from_json(loaded_model_json)
GRU_model.load_weights("q2_data/GRU_model.h5")

In [22]:
# load in the weights and show summary
weights_GRU = GRU_model.get_weights()
GRU_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 512)               857088    
                                                                 
 dense (Dense)               (None, 44)                22572     
                                                                 
 activation (Activation)     (None, 44)                0         
                                                                 
Total params: 879,660
Trainable params: 879,660
Non-trainable params: 0
_________________________________________________________________


Finish the following function that uses a GRU architecture to generate text, given the weights of the GRU model, a text prompt, and the number of characters to return. The function should be completed by **only using numpy functions**. Use your knowledge of how every weight plays its role in the GRU architecture. Do not worry about the weight extraction part, this is already provided for you. The hidden state $h$ is initialized to a vector of zeros. 

The embedding of each character has to be done by a one-hot encoding, where you will need the dictionaries defined in the introduction to go from a character to an index position.

Note: a slightly different version of the GRU is used, where the candidate state $c_t$ is calculated as:

$$
c_t = \text{tanh} \left(W_{hx} x_t \ + \ \Gamma_t^r \odot (W_{hh} h_{t-1}) \ + \ b_h \right)
$$

In [23]:
# Helper function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sample_text_GRU(weights, prompt, N):
    '''
    Uses a pretrained GRU to generate text, starting from a prompt,
    only using the weights and numpy commands
            Parameters:
                    weights (list): Weights of the pretrained GRU model
                    prompt (string): Start of generated sentence
                    N (int): Total length of output sentence
            Returns:
                    output_sentence (string): Text generated by GRU
    '''
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides
    
    # GRU Layer 
    W_ux, W_rx, W_hx = np.split(weights[0].T, 3, axis = 0)
    W_uh, W_rh, W_hh = np.split(weights[1].T, 3, axis = 0)

    bias = np.sum(weights[2], axis=0)
    b_u, b_r, b_h = np.split(np.expand_dims(bias, axis=1), 3)

    # Linear (dense) layer
    W_y = weights[3].T
    b_y = np.expand_dims(weights[4], axis=1)
    
    # Initiate hidden state
    h = np.zeros((W_hh.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    output_sentence = ""
    for i in range(N):
        if i < len(prompt):
            output_sentence += prompt[i]
        else:
            output_sentence += y
        
        # onehot embedding
        x = output_sentence[i]
        x = char_to_indices[x]
        x_one_hot = np.zeros((vocab_size, 1))
        x_one_hot[x] = 1
        x = x_one_hot
        
        # apply GRU
        u = sigmoid(np.dot(W_ux,x) + np.dot(W_uh,h) + b_u)
        r = sigmoid(np.dot(W_rx,x) + np.dot(W_rh,h) + b_r)
        c = np.tanh(np.dot(W_hx,x) + np.dot(r*W_hh,h) + b_h)
        h = (1-u)*c + u*h
        
        y = np.dot(W_y,h) + b_y
        y = np.exp(y) / np.sum(np.exp(y))
        y = np.random.choice(range(vocab_size), p=y.ravel())
        y = indices_to_char[y]
        
    return output_sentence

Test out your function by running the following code cell. Use it as a sanity check that your code is working. The generated text should not be perfect English, but at least you should be able to recognize some words.

In [24]:
print(sample_text_GRU(weights_GRU, 
                      'mrs. oliver looked at herself in the glass. she gave a brief, sideways look',
                      1000))

mrs. oliver looked at herself in the glass. she gave a brief, sideways looked erserout grose, thatiscaveryon with meround was it it. it was to take to want to do ising these time, a terem me. entwansien your very nectly retten know at and to tell of a mind aroustite and tase, to tell you see she interestenting. it or any lifing to allore, appare." "that is the itat of sprofies. to destous to feen a little to itive a it to it. not who wrething it it was a tomestly i mother hain at, and ralensione lofionicg the tage. ot. i moy sot wo lovely at thes it eleehant came she worave it it. caresting him at a lot uf the honaull about the mother of ever the two looked intelesten. i there, treit or live there any more enalooay a prestiof t lisientry into the place. me twly whe knew the atel her some one came to marry. perhaps of edottoction as i tookicatedisame hell or something remember about or the trove of a trout hir lifes by a littre sibte ard the celia looked at alw yed seem dirot and how di

### Problem 2.4: Can Elephants Remember Better?

Perplexity is a measure to quantify how "good" a language model $M$ is, based on a test (or validation) set. The perplexity on a sequence $s$ of characters $a_i$ of size $N$ is defined as:

$$
\text{Perplexity}(M) = M(s)^{(-1/N)} = \left\{p(a_1, \ldots, a_N)\right\}^{(-1/N)} = \left\{p(a_1) \ p(a_2|a_1) \ \ldots \ p(a_N|a_1, \ldots, a_{N-1})\right\}^{(-1/N)}
$$

> The intuition behind this metric is that, if a model assigns a high probability to a test set, it is not surprised to see it (not perplexed by it), which means the model $M$ has a good understanding of how the language works. Hence, a good model has, in theory, a lower perplexity. The exponent $(-1/N)$ in the formula is just a normalizing strategy (geometric average), because adding more characters to a test set would otherwise introduce more uncertainty (i.e. larger test sets would have lower probability). So by introducing the geometric average, we have a metric that is independent of the size of the test set.

When calculating the perplexity, it is important to know that taking the product of a bunch of probabilities will most likely lead to a zero value by the computer. To prevent this, make use of a log-transformation:

$$
\text{Log-Perplexity}(M) = -\frac{1}{N} log\left\{p(a_1, \ldots, a_N)\right\} = -\frac{1}{N} \left\{log \ p(a_1) + \ log \ p(a_2|a_1) + \ \ldots \ + log \  p(a_N|a_1, \ldots, a_{N-1})\right\} 
$$

Don't forget to go back to the normal perplexity after this transformation. 

1. Before calculating the perplexity of a test sequence, start with comparing the outputs of 2.2 and 2.3. Do you see any differences in the generated text of the Vanilla RNN model and the GRU model? Rerun your functions a couple of times (because of stochasticity) and use different prompts. Briefly discuss why you would expect (or not expect) certain differences.

Both models start to become worse when generating far away from the prompt. However, the GRU model seems to be able to remember more of the prompt and/or the training data, such that in all the non-sense characters it generated, there will be a few times that it actually generate some proper English sentences. 

Also, the generated text of RNN become worse sooner than the GRU. 

2. Calculate the perplexity of each language model by using test_text, an unseen extract of the book. Choose the prompt as the first $m$ letters of the test set, where $m$ is a parameter that you can choose yourself. You should be able to reuse the majority of your previous code in this calculation. Discuss your results at the end.

In [31]:
def perplexity_rnn(weights,text,m):
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides

    # First Recurrent Layer 
    W_xh1 = weights[0].T 
    W_h1h1 = weights[1].T 
    b_h1 = np.expand_dims(weights[2], axis=1)

    # Second Recurrent Layer
    W_h1h2 = weights[3].T
    W_h2h2 = weights[4].T
    b_h2 = np.expand_dims(weights[5], axis=1)

    # Linear (dense) layer
    W_h2y = weights[6].T
    b_y = np.expand_dims(weights[7], axis=1)
    
    # Initiate the hidden states
    h1 = np.zeros((W_h1h1.shape[0], 1))
    h2 = np.zeros((W_h2h2.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    log_p = 0

    for i in range(len(text)-1):
        # onehot embedding
        x = text[i]
        x = char_to_indices[x]
        x_one_hot = np.zeros((vocab_size, 1))
        x_one_hot[x] = 1
        x = x_one_hot
        
        # apply RNN
        h1 = np.tanh(np.dot(W_h1h1,h1) + np.dot(W_xh1,x) + b_h1)
        h2 = np.tanh(np.dot(W_h2h2,h2) + np.dot(W_h1h2,h1) + b_h2)
        
        if i < m-1:
            # still prompting, no need to calculate log_p
            continue
        
        # get the probability of the next character
        y = np.dot(W_h2y,h2) + b_y
        y = np.exp(y) / np.sum(np.exp(y))
        next_char = text[i+1]
        next_char = char_to_indices[next_char]
        log_p += np.log(y[next_char])
    
    return np.exp(-log_p/(len(text)-m))

def perplexity_gru(weights,text,m):
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides
    
    # GRU Layer 
    W_ux, W_rx, W_hx = np.split(weights[0].T, 3, axis = 0)
    W_uh, W_rh, W_hh = np.split(weights[1].T, 3, axis = 0)

    bias = np.sum(weights[2], axis=0)
    b_u, b_r, b_h = np.split(np.expand_dims(bias, axis=1), 3)

    # Linear (dense) layer
    W_y = weights[3].T
    b_y = np.expand_dims(weights[4], axis=1)
    
    # Initiate hidden state
    h = np.zeros((W_hh.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    log_p = 0
    
    for i in range(len(text)-1):        
        # onehot embedding
        x = text[i]
        x = char_to_indices[x]
        x_one_hot = np.zeros((vocab_size, 1))
        x_one_hot[x] = 1
        x = x_one_hot
        
        # apply GRU
        u = sigmoid(np.dot(W_ux,x) + np.dot(W_uh,h) + b_u)
        r = sigmoid(np.dot(W_rx,x) + np.dot(W_rh,h) + b_r)
        c = np.tanh(np.dot(W_hx,x) + np.dot(r*W_hh,h) + b_h)
        h = (1-u)*c + u*h
        
        if i < m-1:
            # still prompting, no need to calculate log_p
            continue
        
        y = np.dot(W_y,h) + b_y
        y = np.exp(y) / np.sum(np.exp(y))
        
        next_char = text[i+1]
        next_char = char_to_indices[next_char]
        log_p += np.log(y[next_char])
        
    return np.exp(-log_p/(len(text)-m))


In [42]:
for m in [1, 5, 10, 20, 50, 100]:    
    print("Perplexity with m = {} is RNN = {}, GRU = {}".format(
        m, 
        perplexity_rnn(weights_RNN, test_text, m)[0].round(4),
        perplexity_gru(weights_GRU, test_text, m)[0].round(4)))

Perplexity with m = 1 is RNN = 5.6653, GRU = 3.9346
Perplexity with m = 5 is RNN = 5.665, GRU = 3.9341
Perplexity with m = 10 is RNN = 5.6664, GRU = 3.9357
Perplexity with m = 20 is RNN = 5.6668, GRU = 3.9364
Perplexity with m = 50 is RNN = 5.6572, GRU = 3.9361
Perplexity with m = 100 is RNN = 5.666, GRU = 3.9406


In [45]:
m = 2000
print("Perplexity with m = {} is RNN = {}, GRU = {}".format(
        m, 
        perplexity_rnn(weights_RNN, test_text, m)[0].round(4),
        perplexity_gru(weights_GRU, test_text, m)[0].round(4)))

Perplexity with m = 2000 is RNN = 5.6714, GRU = 3.9435


Observations
* GRU performs much better than RNN, which is expected
* the value of $m$ does not matter too much for both models. This is also expected as well, all the hidden variables is the same regardless of the value of $m$, so the effect of $m$ is minimal on the value of perplexity if $m<<$ size of test_text. 

3. As seen in part 2 and 3 of this problem, the text generation is not perfect. Describe some possible model improvements that could make the quality of the generated text better.

* a larger hidden state dimension
* more layers with residual connection for the hidden states
* apply attention mechanism
  